In [1]:
import pandas as pd
import numpy as np
import calendar
import time
import warnings
import datetime as dt
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('applications data.csv', parse_dates = ['date'])

In [3]:
df['zip5'] = df['zip5'].apply(lambda x: '{0:0>5}'.format(x))
df['dob_datetime'] = pd.to_datetime(df.dob.astype(str))

In [4]:
# Fix frivolous values
# Frivoulous values for this project were values given where data was missing

#ssn 
df["ssn_is_friv"] = np.where(df["ssn"]=="999999999", 1, 0)
df.loc[df.ssn==999999999, 'ssn'] = -df[df.ssn==999999999]['record']
df['ssn'] = df['ssn'].apply(lambda x: '{0:0>9}'.format(x))

# address 
df["address_is_friv"] = np.where(df["address"]=="123 MAIN ST", 1, 0)
df.loc[df.address=='123 MAIN ST', 'address'] = \
    df.loc[df.address=='123 MAIN ST', 'record'].apply(lambda x: str(x)+' RECORD')

# dob
df["dob_is_friv"] = np.where(df["dob"]==19070626, 1, 0)
df.loc[df.dob==19070626, 'dob'] = -df.loc[df.dob==19070626]['record']
df['dob'] = df['dob'].apply(lambda x: '{0:0>8}'.format(x))

# homephone
df["homephone_is_friv"] = np.where(df["homephone"]=="9999999999", 1, 0)
df.loc[df.homephone==9999999999, 'homephone'] = -df.loc[df.homephone==9999999999]['record']
df['homephone'] = df['homephone'].apply(lambda x: '{0:0>10}'.format(x))

In [5]:
# Create different entities
df['name'] = df.firstname + df.lastname
df['fulladdress'] = df.address + df.zip5
df['name_dob'] = df.name + df.dob
df['name_fulladdress'] = df.name + df.fulladdress
df['name_homephone'] = df.name + df.homephone
df['fulladdress_dob'] = df.fulladdress + df.dob
df['fulladdress_homephone'] = df.fulladdress + df.homephone
df['dob_homephone'] = df.dob + df.homephone
df['homephone_name_dob'] = df.homephone + df.name_dob
df['fullname_address_zip_dob'] = df.name + df.fulladdress + df.dob
df['areacode'] = df.homephone.astype(str).str[:3].astype(int)
df['areacode_name_fulladdress'] = df.fulladdress + df.name + df.areacode.astype(str)

In [6]:
# Creates a attribute for which to build features for
for field in list(df.iloc[:, np.r_[3:9, 16:24, 26]].columns):
    df['ssn_' + field] = df.ssn + df[field]

In [7]:
attributes = list(df.iloc[:, np.r_[2, 5, 7, 8, 15:24, 26:42]].columns)

In [8]:
# Creates age-based features for model
ages = pd.to_datetime('10-31-16') - df['dob_datetime']
df['age_days'] = ages.dt.days.astype(int)
df['age_years'] = (ages.dt.days / (365.25)).astype(float)
age_at_apps = df['date'] - df['dob_datetime']
df['age_day_at_app'] = age_at_apps.dt.days.astype(int)
df['age_years_at_app'] = (age_at_apps.dt.days / (365.25)).astype(float)

df['age_days_remove'] = df['age_days']
df['age_years_remove'] = df['age_years']
df['age_app_days_remove'] = df['age_day_at_app']
df['age_app_years_remove'] = df['age_years_at_app']

df.loc[df['age_days_remove'] == 39940, 'age_years_remove'] = 1
df.loc[df['age_years_remove'] == 109.34976043805612, 'age_years_remove'] = 1
df.loc[df['age_app_days_remove'] == 39636, 'age_years_remove'] = 1
df.loc[df['age_app_years_remove'] == 108.51745379876797, 'age_years_remove'] = 1

In [10]:
df1 = df.copy()
final = df.copy()
df1['check_date'] = df1.date
df1['check_record'] = df1.record

In [11]:
start = time.time()
full_entity = []

# Loops through each attribute to create features
for entity in attributes:
    single_entity = ['record', 'age_app_years_remove', 'fulladdress_length']
    st = time.time()
    df_1 = df1[['record', 'date', entity]]
    df_r = df1[['check_record', 'check_date', entity]]
    
    # Keeps the time-order for each feature that is created
    temp = pd.merge(df_1, df_r, left_on = entity, right_on = entity)
    
    # Days since, creates days since record was last seen
    days_since_df = temp[temp.record > temp.check_record][['record', 'date', 'check_date']] \
        .groupby('record')[['date', 'check_date']].last()
    mapper = (days_since_df.date - days_since_df.check_date).dt.days
    final[entity + '_day_since'] = final.record.map(mapper)
    final[entity + '_day_since'].fillna(365, inplace = True)
    single_entity.append(entity + '_day_since')
    print(f'\n{entity}_day_since ----> Done')
    
    # Velocity
    for offset_t in [0,1,3,7,14,30, 90, 180, 365]: 
        count_day_df = temp[(temp.check_date >= (temp.date-dt.timedelta(offset_t))) & (temp.record >= temp.check_record)]
        col_name = f'{entity}_count_{offset_t}'
        mapper2 = count_day_df.groupby('record')[entity].count()
        final[col_name] = final.record.map(mapper2)
        single_entity.append(entity + '_count_' + str(offset_t))
        print(f'{entity}_count_{str(offset_t)} ----> Done')
    if entity in ['ssn', 'address', 'homephone', 'name_fulladdress',
                  'homephone_name_dob', 'fullname_address_zip_dob', 
                  'ssn_lastname', 'ssn_address', 'ssn_zip5', 'ssn_name_dob', 
                  'ssn_name_fulladdress', 'ssn_name_homephone', 'areacode_name_fulladdress',
                  'ssn_areacode_name_fulladdress']:
        full_entity.append(single_entity)
    print(f'Run time for entity {entity} ---------------- {time.time() - st:0.2f}s')
print(f'Total run time: {(time.time()-start) / 60:0.2f}min')


ssn_day_since ----> Done
ssn_count_0 ----> Done
ssn_count_1 ----> Done
ssn_count_3 ----> Done
ssn_count_7 ----> Done
ssn_count_14 ----> Done
ssn_count_30 ----> Done
ssn_count_90 ----> Done
ssn_count_180 ----> Done
ssn_count_365 ----> Done
Run time for entity ssn ---------------- 3.82s

address_day_since ----> Done
address_count_0 ----> Done
address_count_1 ----> Done
address_count_3 ----> Done
address_count_7 ----> Done
address_count_14 ----> Done
address_count_30 ----> Done
address_count_90 ----> Done
address_count_180 ----> Done
address_count_365 ----> Done
Run time for entity address ---------------- 3.85s

dob_day_since ----> Done
dob_count_0 ----> Done
dob_count_1 ----> Done
dob_count_3 ----> Done
dob_count_7 ----> Done
dob_count_14 ----> Done
dob_count_30 ----> Done
dob_count_90 ----> Done
dob_count_180 ----> Done
dob_count_365 ----> Done
Run time for entity dob ---------------- 24.39s

homephone_day_since ----> Done
homephone_count_0 ----> Done
homephone_count_1 ----> Done
home

ssn_name_dob_count_14 ----> Done
ssn_name_dob_count_30 ----> Done
ssn_name_dob_count_90 ----> Done
ssn_name_dob_count_180 ----> Done
ssn_name_dob_count_365 ----> Done
Run time for entity ssn_name_dob ---------------- 3.49s

ssn_name_fulladdress_day_since ----> Done
ssn_name_fulladdress_count_0 ----> Done
ssn_name_fulladdress_count_1 ----> Done
ssn_name_fulladdress_count_3 ----> Done
ssn_name_fulladdress_count_7 ----> Done
ssn_name_fulladdress_count_14 ----> Done
ssn_name_fulladdress_count_30 ----> Done
ssn_name_fulladdress_count_90 ----> Done
ssn_name_fulladdress_count_180 ----> Done
ssn_name_fulladdress_count_365 ----> Done
Run time for entity ssn_name_fulladdress ---------------- 3.53s

ssn_name_homephone_day_since ----> Done
ssn_name_homephone_count_0 ----> Done
ssn_name_homephone_count_1 ----> Done
ssn_name_homephone_count_3 ----> Done
ssn_name_homephone_count_7 ----> Done
ssn_name_homephone_count_14 ----> Done
ssn_name_homephone_count_30 ----> Done
ssn_name_homephone_count_90 ----

In [12]:
start = time.time()
# Creates Relative Velocity features, comparing the ratio between recent time frames to longer period of time 
for att in attributes:
    for d in ['0', '1', '3']:
        for dd in ['3', '7', '14', '30', '90', '180', '365']:
            final[att + '_count_' + d + '_by_' + dd] \
                = final[att + '_count_' + d] / (final[att + '_count_' + dd] / float(dd))
print(f'Total run time: {time.time() - start:0.2f}s')

Total run time: 25.93s


In [13]:
import featuretools as ft

count = 0
start = time.time()

# Automated feature engineering technique to gather ratios between different features
for i in full_entity:
    single = time.time()
    subset = final[i]
    es = ft.EntitySet(id = 'Numeric Data')
    es.entity_from_dataframe(entity_id = 'Subset', dataframe = subset, index = 'record')

    # Run deep feature synthesis with transformation primitives
    features, feature_names = ft.dfs(entityset = es, target_entity = 'Subset',
                                          trans_primitives = ['add_numeric', 'divide_numeric'], 
                                          verbose=True, n_jobs = -1, max_depth = 2)
    features = features.reset_index()
    features = features.drop(columns = full_entity[count])
    final = pd.concat([final, features], axis = 1)
    print(f'Total run time: {time.time() - single:0.2f}s')
    count += 1
print(f'Total run time: {time.time() - start:0.2f}s')

Built 210 features
EntitySet scattered to 12 workers in 6 seconds
Elapsed: 00:14 | Progress: 100%|██████████
Total run time: 42.54s
Built 210 features
EntitySet scattered to 12 workers in 6 seconds
Elapsed: 00:15 | Progress: 100%|██████████
Total run time: 57.55s
Built 210 features
EntitySet scattered to 12 workers in 6 seconds
Elapsed: 00:15 | Progress: 100%|██████████
Total run time: 71.55s
Built 210 features
EntitySet scattered to 12 workers in 5 seconds
Elapsed: 00:15 | Progress: 100%|██████████
Total run time: 84.16s
Built 210 features
EntitySet scattered to 12 workers in 6 seconds
Elapsed: 00:15 | Progress: 100%|██████████
Total run time: 92.46s
Built 210 features
EntitySet scattered to 12 workers in 5 seconds
Elapsed: 00:15 | Progress: 100%|██████████
Total run time: 104.78s
Built 210 features
EntitySet scattered to 12 workers in 5 seconds
Elapsed: 00:16 | Progress: 100%|██████████
Total run time: 115.93s
Built 210 features
EntitySet scattered to 12 workers in 6 seconds
Elapsed:

In [14]:
stored_attributes_length_names = ['name_length']
for i in attributes: 
    df[i + '_length'] = df[i].str.len()
    stored_attributes_length_names.append(i + 'length')

In [15]:
oot = final.loc[final['date'] > '10-31-16']
train_test = final.loc[final['date'] <= '10-31-16']

In [16]:
# Risk Table


# Creates a target-encoding variable for categorical day of week
# Avoids overfitting via exponential smoothing
train_test['dow'] = train_test.date.apply(lambda x: calendar.day_name[x.weekday()])
final['dow'] = final.date.apply(lambda x: calendar.day_name[x.weekday()])

c = 4
nmid = 100
y_avg = train_test['fraud_label'].mean()
y_dow = train_test.groupby('dow')['fraud_label'].mean()
num = train_test.groupby('dow').size()
y_dow_smooth = y_avg + (y_dow - y_avg)/(1 + np.exp(-(num - nmid)/ c))
final['dow_risk'] = final.dow.map(y_dow_smooth)

In [17]:
train_test['year_dob'] = train_test.dob_datetime.dt.year
final['year_dob'] = final.dob_datetime.dt.year

c = 4
nmid = 100
y_avg = train_test['fraud_label'].mean()
y_dow = train_test.groupby('year_dob')['fraud_label'].mean()
num = train_test.groupby('year_dob').size()
y_dow_smooth = y_avg + (y_dow - y_avg)/(1 + np.exp(-(num - nmid)/ c))
final['year_dob_risk'] = final.year_dob.map(y_dow_smooth)

In [18]:
train_test['month_dob'] = train_test.dob_datetime.dt.month
final['month_dob'] = final.dob_datetime.dt.month

c = 4
nmid = 100
y_avg = train_test['fraud_label'].mean()
y_dow = train_test.groupby('month_dob')['fraud_label'].mean()
num = train_test.groupby('month_dob').size()
y_dow_smooth = y_avg + (y_dow - y_avg)/(1 + np.exp(-(num - nmid)/ c))
final['month_dob_risk'] = final.month_dob.map(y_dow_smooth)

In [19]:
train_test['day_dob'] = train_test.dob_datetime.dt.day
final['day_dob'] = final.dob_datetime.dt.day

c = 4
nmid = 100
y_avg = train_test['fraud_label'].mean()
y_dow = train_test.groupby('day_dob')['fraud_label'].mean()
num = train_test.groupby('day_dob').size()
y_dow_smooth = y_avg + (y_dow - y_avg)/(1 + np.exp(-(num - nmid)/ c))
final['day_dob_risk'] = final.day_dob.map(y_dow_smooth)

In [20]:
# Risk Table
train_test['dow_dob'] = train_test.dob_datetime.apply(lambda x: calendar.day_name[x.weekday()])
final['dow_dob'] = final.dob_datetime.apply(lambda x: calendar.day_name[x.weekday()])

c = 4
nmid = 100
y_avg = train_test['fraud_label'].mean()
y_dow = train_test.groupby('dow_dob')['fraud_label'].mean()
num = train_test.groupby('dow_dob').size()
y_dow_smooth = y_avg + (y_dow - y_avg)/(1 + np.exp(-(num - nmid)/ c))
final['dow_dob_risk'] = final.dow_dob.map(y_dow_smooth)

In [21]:
len(final.columns) + 7

3542

In [22]:
final = final.replace(np.inf, 0)
final = final.replace(-np.inf, 0)

In [23]:
#final_stats = final.describe().transpose()

In [24]:
#final_stats.to_csv('IdentityFraudVariablesStatistics.csv')

In [25]:
#final = final.loc[:,~final.columns.duplicated()]
#final.to_hdf(r'ApplicationsDataVariables.h5', key='stage', mode='w')
#final.to_csv('ApplicationsDataVariables.csv')

In [26]:
final = final.loc[:,~final.columns.duplicated()]
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

final_subset = final[final['date'] < '11-01-16']
final_subset = final_subset[final_subset['date'] > '01-14-16']

final_numeric =final_subset.select_dtypes(include=numerics)

In [27]:
final_numeric.set_index('record', inplace = True)

In [29]:
KSFDR = pd.DataFrame(columns = ['Variable', 'KS', 'FDR'])
KSFDR['Variable'] = list(final_numeric.columns ) 

In [30]:
from scipy import stats as sps
i = 0

# Finds the KS-score of each variable
goods = final_numeric.loc[final_numeric['fraud_label'] == 0]
bads = final_numeric.loc[final_numeric['fraud_label'] == 1]
for column in final_numeric:
    KSFDR['KS'][i] = sps.ks_2samp(goods[column],bads[column])[0]
    i = i+1

In [31]:
j = 0
topRows = int(round(len(final_numeric)*0.03))

# Finds the Fraud Detection Rate for each single variable
for column in final_numeric:    
    temp = final_numeric[[column,'fraud_label']].copy()
    if column == 'fraud_label':
        temp['outcome']= temp.iloc[:,0]
        temp = temp.iloc[:,1:]
        column = 'outcome'
    temp0 = temp.sort_values(column,ascending=False)
    temp1 = temp0.head(topRows)
    temp2 = temp0.tail(topRows)
    numbads = sum(temp0.loc[:, 'fraud_label'])
    needed1 = temp1.loc[:,'fraud_label']
    needed2 = temp2.loc[:,'fraud_label']
    FDR1 = sum(needed1)/numbads
    FDR2 = sum(needed2)/numbads
    FDRate = np.maximum(FDR1,FDR2)
    KSFDR.loc[j, 'FDR'] = FDRate
    j = j + 1


In [32]:
# Rank orders the KS and FDR univariate features 
KSFDR['rank_ks'] = KSFDR['KS'].rank(ascending = True)
KSFDR['rank_FDR'] = KSFDR['FDR'].rank(ascending = True)
KSFDR['average_rank'] = (KSFDR['rank_ks'] + KSFDR['rank_FDR']) / 2
KSFDR.sort_values(by=['average_rank'], ascending=False, inplace=True)

In [33]:
#KSFDR.to_csv('VariablePerformanceRoughDraft.csv')

In [34]:
# Keeps the Top 100 features based on the highest average KS and FDR rate
final.set_index('record', inplace = True)
reduced = final[list(KSFDR['Variable'][:100])]
reduced['date'] = df['date']

In [38]:
reduced['date'] = final['date']

In [39]:
reduced.to_csv('ApplicationsDataAfterFeatureSelection.csv')